## Connect to your GDrive

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/MyDrive


### Install `mimikit`

In [ ]:

!pip install mimikit==0.2.0

### import and arguments

In [ ]:
import mimikit as mmk

# DATA

# list of files or directories to use as data
sources = ['./data']
# audio sample rate
sr = 22050
# the size of the stft
n_fft = 2048
# hop_length of the
hop_length = n_fft // 4

# NETWORK

# the number of layers determines 'how much past' is used to predict the next future step
# here you can make blocks of layers by specifying a tuple of integers, e.g. (2, 3, 2)
n_layers = (3, )
# how many parameters pro layer (must be divisible by 2)
gate_dim = 1024
# this multiplies the number of parameters used in the input & output layers.
n_heads = 4

# OPTIMIZATION

# how many epochs should we train for
max_epochs = 50
# how many examples are used pro training steps
batch_size = 16
# how long are the examples in each training step
# must be bigger than the network's receptive field
batch_seq_length = sum(2**n for n in n_layers) * 2
# the learning rate
max_lr = 5e-4
# betas control how fast the network changes its 'learning course'.
# generally, betas should be close but smaller than 1. and be balanced with the batch_size :
# the smaller the batch, the higher the betas 'could be'.
betas = (0.9, 0.93)

# MONITORING

# how often should the network generate during training
every_n_epochs = 2
# how many examples from random prompts should be generated
n_examples = 3
# how many steps (1 step = 1 fft frame) should be generated
n_steps = 1000

# make sure batch are long enough
rf = sum(2**n for n in n_layers) * 2
assert batch_seq_length > rf, f"batch_seq_length ({batch_seq_length}) needs to be bigger than the receptive field ({rf})"
print("Ok!")



### create the data

In [ ]:
schema = mmk.FreqNet.schema(sr, n_fft=n_fft, hop_length=hop_length,
                            segment_labels=False, files_labels=False)

db_path = 'freqnet-demo.h5'
db = mmk.Database.create(db_path, sources, schema)
print("Successfully created the db.")



### create network and train

In [ ]:
net = mmk.FreqNet(
    **mmk.FreqNet.dependant_hp(db),
    n_layers=n_layers,
    gate_dim=gate_dim,
    groups=2,
    n_input_heads=n_heads,
    n_output_heads=n_heads,
    batch_size=batch_size,
    batch_seq_length=batch_seq_length,
    max_lr=max_lr,
    div_factor=5,
    betas=betas,

)
print(net.hparams)

dm = mmk.DataModule(net, db, splits=tuple())

cb = mmk.GenerateCallBack(every_n_epochs,
                          indices=[None]*n_examples,
                          n_steps=n_steps,
                          play_audios=True,
                          plot_audios=True)

trainer = mmk.get_trainer(root_dir=None,
                          max_epochs=max_epochs,
                          callbacks=[cb],
                          checkpoint_callback=False)

trainer.fit(net, datamodule=dm)



----------------------------

<img src="https://ktonal.com/k-circle-bw.png" alt="logo" width="75"/>